In [1]:
import pandas as pd
import time

import json
import httplib2
import sys
import codecs
       
import re

import numpy as np
import math
import ast

import warnings
warnings.filterwarnings("ignore")

import json
#!pip install geog
import geog
import shapely.geometry
from shapely.geometry import Point

In [2]:
df = pd.read_csv('new_venues_clean.csv', encoding = "ISO-8859-1") 

In [3]:
# Test
p = shapely.geometry.Point([-118.24368, 34.05223])
n_points = 4
d = 30 * 1000  # meters

angles = np.linspace(0, 360, n_points)
polygon = geog.propagate(p, angles, d)
box = shapely.geometry.Polygon(polygon)
#print (type (box))
#point = Point(-118.4022254,33.94113193)

point = Point(-73.935242, 40.730610)

point.within(box)

#print(json.dumps(shapely.geometry.mapping(shapely.geometry.Polygon(polygon))))

False

In [4]:
def list_of_centroids(x_0,y_0,x_f,y_f,r_a):
    #Euclidean distance from origin to Destination
    d_t_1 = ((y_f-y_0)**2+(x_f-x_0)**2)**.5
    print ('Euclidean Distance between Origin and Destination', d_t_1)
    
    #Haversine distance from origin to Destination
    dlat = math.radians(x_f-x_0)
    dlon = math.radians(y_f-y_0)
    radius = 6371 # km
    a = math.sin(dlat/2) * math.sin(dlat/2) + math.cos(math.radians(x_f)) \
        * math.cos(math.radians(x_0)) * math.sin(dlon/2) * math.sin(dlon/2)
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1-a))
    d_t = radius * c
    print ('Haversine Distance between Origin and Destination', d_t)
    
    #actual number of points needed
    n = round(d_t/r_a)-1
    print ('Number of points', n)
    
    x_delta = (x_f - x_0)/n
    y_delta = (y_f - y_0)/n

    #in future iterations this could be points along the route
    list_of_centroids = [(x_0+i*x_delta, y_0+i*y_delta) for i in range(1,n)]
    print ('List of Centroids\n', list_of_centroids)
    return (list_of_centroids)

In [5]:
def poi_inside_box(centroid_list,r_a,polygon_points):
    #list of tuples in the form of (x_min, x_max, y_min, y_max)
    #bounds_of_each_box = [(x-r_a,x+r_a,y-r_a,y+r_a)  for (x,y) in list_of_centroids]
    d = r_a * 1000  # meters
    angles = np.linspace(0, 360, polygon_points)
    global df
    
    df['in_the_box'] = False
    for i,row in df.iterrows():
        for (x,y) in centroid_list:
            p = shapely.geometry.Point([y, x])
            polygon = geog.propagate(p, angles, d)
            box = shapely.geometry.Polygon(polygon)
            point = Point(df['lng'][i], df['lat'][i])
            if point.within(box):
                df['in_the_box'][i] = point.within(box)
                break

    
   # print ("Bounds of Square\n", bounds_of_each_box)
    
#after this you would loop through your poi (or use sapply or lapply (i can never keep them straight)
#for each poi you could simply check if it's coordinates are within the x AND y bounds for ANY of teh boxes we calculated

In [6]:
# Origin and Destination
#origin = [34.05223, -118.24368]
#destination = [37.77493, -122.41942]

x_0 = 34.05223
y_0 = -118.24368
x_f = 37.77493
y_f = -122.41942

print ("Origin Lat/Long", x_0, y_0 )
print ("Destination Lat/Long", x_0, y_0 )
#approximate distance between points (aka "radius")
r_a = 20
print ("Radius", r_a )

centroids = list_of_centroids(x_0,y_0,x_f,y_f,r_a)


#poi_inside_box()

Origin Lat/Long 34.05223 -118.24368
Destination Lat/Long 34.05223 -118.24368
Radius 20
Euclidean Distance between Origin and Destination 5.59422021711695
Haversine Distance between Origin and Destination 559.1229009538062
Number of points 27
List of Centroids
 [(34.190107777777776, -118.39833703703704), (34.32798555555556, -118.55299407407408), (34.46586333333333, -118.7076511111111), (34.60374111111111, -118.86230814814814), (34.74161888888889, -119.01696518518519), (34.87949666666667, -119.17162222222223), (35.01737444444444, -119.32627925925925), (35.155252222222224, -119.48093629629629), (35.29313, -119.63559333333333), (35.43100777777778, -119.79025037037037), (35.56888555555555, -119.94490740740741), (35.706763333333335, -120.09956444444444), (35.84464111111111, -120.25422148148148), (35.98251888888889, -120.40887851851852), (36.120396666666664, -120.56353555555556), (36.258274444444446, -120.71819259259259), (36.39615222222222, -120.87284962962963), (36.53403, -121.0275066666666

In [7]:
polygon_points = 4
poi_inside_box(centroids,r_a,polygon_points)

    

In [8]:
df[df['in_the_box'] == True]

,Unnamed: 0,Unnamed: 0.1,name,id,category,address,lat,lng,city,state,price_tier,serves,bucket,big_bucket,like_counts,rating,image_url,in_the_box
1186,1248,1248,Madera,4a71f2c0f964a520f2d91fe3,'new american restaurant',2825 Sand Hill Rd,37.419361,-122.212178,Menlo Park,CA,3,"Dessert, Lunch & more",3.0,1,167,8.3,https://fastly.4sqi.net/img/general/300x500/10...,True
1188,1250,1250,Wakuriya,4b108551f964a520277223e3,'japanese restaurant',115 De Anza Blvd,37.521101,-122.337092,San Mateo,CA,4,Dinner,9.0,1,35,8.6,https://fastly.4sqi.net/img/general/300x500/15...,True
1189,1251,1251,Dutch Goose,40eb3d00f964a520280a1fe3,'burger joint',3567 Alameda de Las Pulgas,37.431971,-122.201592,Menlo Park,CA,1,"Happy Hour, Brunch & more",0.0,1,239,8.5,https://fastly.4sqi.net/img/general/300x500/-F...,True
1190,1252,1252,Rainbow Pizza,4a30bc2df964a52088991fe3,'pizza place',112 De Anza Blvd,37.521329,-122.337338,San Mateo,CA,2,"Dinner, Lunch & more",5.0,1,43,8.4,https://fastly.4sqi.net/img/general/300x500/40...,True
1193,1255,1255,Portola Kitchen,50060bfbe4b0d5ab11480f62,'italian restaurant',3130 Alpine Rd,37.401415,-122.193228,Portola Valley,CA,4,"Dinner, Lunch & more",7.0,1,37,8.2,https://fastly.4sqi.net/img/general/300x500/13...,True
1196,1258,1258,Dianda's Italian American Pastry - San Mateo,4b243ac7f964a5203b6424e3,'bakery',117 Crystal Springs Shopping Center,37.521082,-122.337113,San Mateo,CA,1,"Lunch, Brunch & more",1.0,1,12,8.3,https://fastly.4sqi.net/img/general/300x500/28...,True
1197,1259,1259,Avanti Pizza,4502cef0f964a520d1381fe3,'pizza place',3536 Alameda de Las Pulgas,37.431990,-122.202241,Menlo Park,CA,1,NaN,5.0,1,9,8.3,https://fastly.4sqi.net/img/general/300x500/11...,True
1198,1260,1260,Flea Street Cafe,4ac7f846f964a520ecba20e3,'new american restaurant',3607 Alameda de Las Pulgas,37.431774,-122.201329,Menlo Park,CA,4,Dinner,3.0,1,45,8.3,https://fastly.4sqi.net/img/general/300x500/p5...,True
1208,1270,1270,Restaurant Mitsunobu,4ea4c815e3008f7d9312dfc4,'japanese restaurant',325 Sharon Park Dr Ste A2,37.424049,-122.196618,Menlo Park,CA,4,Lunch & Dinner,9.0,1,24,8.0,https://fastly.4sqi.net/img/general/300x500/bM...,True
1217,1279,1279,Mademoiselle Colette,56194b44498ed60329b635fa,'bakery',816 Santa Cruz Ave,37.450872,-122.185341,Menlo Park,CA,1,"Happy Hour, Lunch & more",1.0,1,76,8.8,https://fastly.4sqi.net/img/general/300x500/56...,True


In [11]:
df.drop(['Unnamed: 0','Unnamed: 0.1'],axis =1, inplace=True)

In [12]:
df

,name,id,category,address,lat,lng,city,state,price_tier,serves,bucket,big_bucket,like_counts,rating,image_url,in_the_box
0,Hopdoddy Burger Bar,55cd5356498ebfb0cba0dcb2,'burger joint',830 S Pacific Coast Hwy Ste 116,33.927394,-118.396044,El Segundo,CA,1,"Happy Hour, Brunch & more",0.0,1,116,8.5,https://fastly.4sqi.net/img/general/300x500/17...,False
1,In-N-Out Burger,55d631df498ed45ed4a11721,'burger joint',600 N. Sepulveda Blvd,33.923919,-118.395773,El Segundo,CA,1,"Dinner, Lunch & more",0.0,1,143,8.4,https://fastly.4sqi.net/img/general/300x500/33...,False
2,Bread & Butter,571f9a21498edb90145573ba,'diner',630 North Sepulveda,33.924438,-118.395682,El Segundo,CA,1,"Lunch, Brunch & more",3.0,1,6,7.8,https://fastly.4sqi.net/img/general/300x500/50...,False
3,Jersey Mike's Subs,5b588428b77c770039515458,'sandwich place',"1 World Way-Los Angeles International Airport,...",33.938398,-118.405088,Los Angeles,CA,1,NaN,0.0,1,0,7.5,https://lh3.googleusercontent.com/p/AF1QipPxDr...,False
4,Jersey Mike's Subs,51ddafe7498e3265d612ee18,'sandwich place',530 A N. Sepulveda Blvd.,33.922938,-118.395819,El Segundo,CA,1,"Dinner, Lunch & more",0.0,1,20,7.5,https://fastly.4sqi.net/img/general/300x500/13...,False
5,El Pollo Loco,4b6a2104f964a52035c92be3,'fast food restaurant',630 N Sepulveda Blvd,33.924537,-118.395912,El Segundo,CA,1,"Dinner, Lunch & more",0.0,1,29,7.3,https://fastly.4sqi.net/img/general/300x500/U4...,False
6,Petit Cafe,4af8731bf964a5205b0d22e3,'falafel restaurant',630 N Sepulveda Blvd,33.924541,-118.395686,El Segundo,CA,1,"Dinner, Lunch & more",8.0,1,9,7.2,https://fastly.4sqi.net/img/general/300x500/JE...,False
7,SUBWAY,4fc585e9e4b01c7a894b9268,'sandwich place',630 N Sepulveda Blvd,33.924353,-118.395875,El Segundo,CA,1,"Dinner, Breakfast & more",0.0,1,4,7.2,https://fastly.4sqi.net/img/general/300x500/50...,False
8,IHOP,4b6884e7f964a520857d2be3,'breakfast spot',755 N. Sepulveda Blvd.,33.926210,-118.396575,El Segundo,CA,2,"Dinner, Breakfast & more",0.0,1,90,6.9,https://fastly.4sqi.net/img/general/300x500/2k...,False
9,The Habit Burger,5606fe0c498ecb20435f1bd4,'burger joint',Terminal 6,33.940871,-118.401965,Los Angeles,CA,1,"Dinner, Lunch & more",0.0,1,36,7.1,https://fastly.4sqi.net/img/general/300x500/11...,False


In [13]:
df.to_csv('new_venues_clean.csv')